In [4]:
!pip install git+https://github.com/yidinghao/interpreting-nlp#egg=interpret_nlp
!pip install yattag

  Cloning https://github.com/yidinghao/interpreting-nlp to /tmp/pip-install-mwmybrod/interpret-nlp_edec3213e28747bc8545451c20ac9700
  Running command git clone --filter=blob:none --quiet https://github.com/yidinghao/interpreting-nlp /tmp/pip-install-mwmybrod/interpret-nlp_edec3213e28747bc8545451c20ac9700
  Resolved https://github.com/yidinghao/interpreting-nlp to commit b6bf81e2e52970ffe4db32cade8c9c3d48d1cd3b
  Preparing metadata (setup.py) ... done
  Created wheel for interpret_nlp: filename=interpret_nlp-0.1-py3-none-any.whl size=14664 sha256=3e9ddc6e0917b4f7689dc1a70cc6de4d6de1bb81e5b7cb2f6699da5db1fd291b
  Stored in directory: /tmp/pip-ephem-wheel-cache-szac3tcj/wheels/c0/d4/0f/9d21571e687d4e3dcf61b6538dedff103fdc48bad18ca6c357
Successfully built interpret_nlp


In [5]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from interpret_nlp.modules.lrp_modules import LRPLinear, LRPGRU

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
TRAIN_PATH = "/content/drive/MyDrive/Workspace/Copy of train.pkl"
TEST_PATH = "/content/drive/MyDrive/Workspace/Copy of test.pkl"
RS_TRAIN_PATH = "/content/drive/MyDrive/Workspace/Copy of rs_train_embeddings.pkl"
RS_TEST_PATH = "/content/drive/MyDrive/Workspace/Copy of rs_test_embeddings.pkl"

In [10]:
with open(TRAIN_PATH, 'rb') as f:
    train = pickle.load(f)

with open(TEST_PATH, 'rb') as f:
    test = pickle.load(f)

with open(RS_TRAIN_PATH, 'rb') as f:
    rs_train_embeddings = pickle.load(f)

with open(RS_TEST_PATH, 'rb') as f:
    rs_test_embeddings = pickle.load(f)

LRP Implementaion

In [15]:
train.head()

,tweet,label,tweet_with_out_emojis,cleaned_data,Light Stemming,Root Stemming
1426,شايف نفسه الحمار مع انه كان احتياط الموسم الما...,1,شايف نفسه الحمار مع انه كان احتياط الموسم الما...,شايف الحمار انه احتياط الموسم الماضي اليوفي اد...,شايف حمار انه احتياط موسم ماضي يوفي اداره ريال...,شيف حمر انه حيط وسم اضي يوف دره ريل امر غرب هج...
496,: هزق البلد كلها ..مرتضي محدش عارف يوقفه ..ل...,1,: هزق البلد كلها ..مرتضي محدش عارف يوقفه ..ل...,هزق البلد مرتضي محدش عارف يوقفه لا وزير داخليه...,هزق بلد مرتضي محدش عارف يوقفه لا وزير داخليه ش...,هزق بلد رضي حدش عرف وقف لا وزر دخل شبب حتي بتع...
3464,استفتاء : عندما تنتهي الاوليمبياد وتعود بعثتنا...,0,استفتاء : عندما تنتهي الاوليمبياد وتعود بعثتنا...,استفتاء تنتهي الاوليمبياد وتعود بعثتنا بالسلام...,استفتاء تنتهي اوليمبياد وتعود بعثت سلامه ميدال...,فاء نهي اوليمبياد تعد بعث سلم يدل حصل وجه نظر
7073,Alhashemلا يوجد مخرج لنفط داعش سوي تركيا.هكذا ...,1,Alhashemلا يوجد مخرج لنفط داعش سوي تركيا.هكذا ...,لا يوجد مخرج لنفط داعش سوي تركياهكذا تقول الجغ...,لا يوجد مخرج لنفط داعش سوي تركياهكذا تقول جغرا...,لا وجد خرج نفط دعش سوي تركياهكذا تقل جغراف سؤل...
758,ما احب اركب جاهل معاي السياره عادي اتوتر طول م...,7,ما احب اركب جاهل معاي السياره عادي اتوتر طول م...,ما احب اركب جاهل معاي السياره عادي اتوتر طول م...,ما احب اركب جاهل معاي سياره عادي اتوتر طول ما ...,ما احب ركب جهل عاي سير عدي وتر طول ما انا اسق ...


In [16]:
len(rs_train_embeddings[0])

768

In [17]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [18]:
X_train_gru_global = rs_train_embeddings.reshape(rs_train_embeddings.shape[0], 1, rs_train_embeddings.shape[1])
X_test_gru_global = rs_test_embeddings.reshape(rs_test_embeddings.shape[0], 1, rs_test_embeddings.shape[1])

In [19]:
# handle out of vocab words
tokenizer = Tokenizer(oov_token='')
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train["Root Stemming"])
tokenizer.word_index[''] = 0

In [20]:
# showing the words with the max index
word_with_max_idx = max([(k, v) for k, v in tokenizer.word_index.items()], key = lambda x:x[1])
# showing the words with the min index
word_with_min_idx = min([(k, v) for k, v in tokenizer.word_index.items()], key = lambda x:x[1])
idx_of_UNK = tokenizer.word_index['']
print(word_with_max_idx, word_with_min_idx, idx_of_UNK)

('هتد', 8644) ('', 0) 0


In [21]:
train_sequences = tokenizer.texts_to_sequences(train["Root Stemming"])
test_sequences = tokenizer.texts_to_sequences(test["Root Stemming"])

In [22]:
class myDataset(Dataset):
  def __init__(self, train_sequences, labels):
      # save data
      self.train_sequences = train_sequences
      self.labels = labels

  def __getitem__(self, index):
      # retrieve data
      train_sequence = self.train_sequences[index]
      label = self.labels[index]

      return (torch.tensor(train_sequence), torch.tensor(label))

  def __len__(self):
      return len(self.labels)

In [23]:
# class LSTMEmotionModel(nn.Module):
#   def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
#     super(LSTMEmotionModel, self).__init__()
#     self.embedding = nn.Embedding(vocab_size, embedding_dim)
#     self.hidden_size = hidden_size
#     self.num_layers = num_layers
#     self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
#     self.fc = nn.Linear(hidden_size, num_classes)

#   def forward(self, x):
#     embedded = self.embedding(x)
#     h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#     c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#     out, _ = self.lstm(embedded, (h0, c0))
#     out = self.fc(out[:, -1, :])  # Take the last time step's output
#     return out

In [362]:
class GRUClassifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
    super(GRUClassifier, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    # initial_embeddings = torch.tensor(rs_train_embeddings)
    # self.embedding = nn.Embedding.from_pretrained(initial_embeddings, freeze=False)
    self.gru = LRPGRU(embedding_dim, hidden_size, bidirectional=True)
    self.linear = LRPLinear(hidden_size, 8)

  def forward(self, x: torch.LongTensor) -> torch.FloatTensor:
    embeddings = self.embedding(x)
    hidden = self.gru(embeddings)[1][0]
    hidden = hidden.view(-1, hidden_size)
    out = self.linear(hidden)
    return out

In [363]:
x_train.shape

(8052, 50)

In [364]:
maxlen = 50
x_train = sequence.pad_sequences(train_sequences, maxlen=maxlen)
x_test = sequence.pad_sequences(test_sequences, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8052, 50)
x_test shape: (2013, 50)


In [365]:
customDataset = myDataset(x_train, y_train)
# split the list into training and testing sets
train_data, val_data = train_test_split(customDataset, train_size=0.95)

In [366]:
batch_size = 128
# creating training and validation data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(val_data, batch_size=batch_size)

In [367]:
# Define hyperparameters
hidden_size = 128
num_layers = 2
num_classes = 8
# vocab_size = len(rs_train_embeddings)  # Size of your vocabulary
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = rs_train_embeddings.shape[1]  # Size of your embedding vectors

In [368]:
rs_train_embeddings.shape

(8052, 768)

In [369]:
# # Initialize the model
# model = LSTMEmotionModel(vocab_size, embedding_dim, hidden_size, num_layers, num_classes)

In [370]:
# Initialize the model
model = GRUClassifier(vocab_size, embedding_dim, hidden_size, num_layers)

In [371]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [372]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
  model.train()
  for inputs, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(inputs)
    labels = torch.argmax(labels, dim=1)  # Convert one-hot labels to integer labels
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # Print training progress
    print(f'Loss: {loss.item():.4f}')

Loss: 2.1170
Loss: 2.0755
Loss: 2.0509
Loss: 2.0053
Loss: 1.9920
Loss: 2.0003
Loss: 1.9581
Loss: 1.9372
Loss: 1.9445
Loss: 1.9929
Loss: 1.9428
Loss: 1.9497
Loss: 1.8569
Loss: 1.8537
Loss: 1.8360
Loss: 1.8101
Loss: 1.7981
Loss: 1.8407
Loss: 1.7920
Loss: 1.8250
Loss: 1.8574
Loss: 1.8671
Loss: 1.8509
Loss: 1.8265
Loss: 1.8409
Loss: 1.7545
Loss: 1.7117
Loss: 1.8158
Loss: 1.7231
Loss: 1.6849
Loss: 1.6498
Loss: 1.6456
Loss: 1.7342
Loss: 1.6331
Loss: 1.6849
Loss: 1.6731
Loss: 1.6588
Loss: 1.6819
Loss: 1.6618
Loss: 1.6060
Loss: 1.6170
Loss: 1.5725
Loss: 1.5879
Loss: 1.5881
Loss: 1.5800
Loss: 1.5820
Loss: 1.6190
Loss: 1.4612
Loss: 1.6709
Loss: 1.5756
Loss: 1.4351
Loss: 1.4434
Loss: 1.4663
Loss: 1.6832
Loss: 1.5868
Loss: 1.4544
Loss: 1.5016
Loss: 1.5745
Loss: 1.5696
Loss: 1.5380
Loss: 1.3471
Loss: 1.2737
Loss: 1.1685
Loss: 1.1580
Loss: 1.2597
Loss: 1.1675
Loss: 1.1644
Loss: 1.1625
Loss: 1.2283
Loss: 1.1884
Loss: 1.1483
Loss: 1.1357
Loss: 1.1393
Loss: 1.2567
Loss: 1.2524
Loss: 1.1071
Loss: 1.0125

In [373]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for inputs, labels in test_loader:
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      labels = torch.argmax(labels, dim=1)  # Convert one-hot labels to integer labels
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

Test Accuracy: 62.53%


In [374]:
# Save the model state dictionary
torch.save(model.state_dict(), 'model_state.pth')

In [433]:
# # Initialize the model architecture
# model = LSTMEmotionModel(vocab_size, embedding_dim, hidden_size, num_layers, num_classes)

# # Load the saved model state dictionary
model.load_state_dict(torch.load('model_state.pth'))
model.eval()  # Set the model to evaluation mode

GRUClassifier(
  (embedding): Embedding(8645, 768)
  (gru): LRPGRU(768, 128, batch_first=True, bidirectional=True)
  (linear): LRPLinear(in_features=128, out_features=8, bias=True)
)

In [434]:
# # Save the entire model
# torch.save(model, 'complete_model.pth')

In [435]:
# # Load the entire model
# loaded_model = torch.load('complete_model.pth')
# loaded_model.eval()  # Set the loaded model to evaluation mode

In [436]:
def transform(text):
  tokinized = tokenizer.texts_to_sequences(text)
  seq = sequence.pad_sequences(tokinized, maxlen=maxlen)
  print(seq.shape)
  return torch.LongTensor(seq)

In [437]:
x_test[0].reshape(1,-1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 626, 242, 994,   7,  38, 767]],
      dtype=int32)

In [438]:
x = torch.LongTensor(x_test[0].reshape(1,-1))
print(x.shape)
y = model(x).detach().numpy()

torch.Size([1, 50])


In [439]:
x.shape

torch.Size([1, 50])

In [440]:
# LRP forward
model.linear.attr()
model.gru.attr()

In [441]:
embeddings = model.embedding(x)
gru_out = model.gru(embeddings.detach().numpy())  # 1, 8, 120

In [442]:
gru_out.shape

(1, 50, 256)

In [443]:
h_final = np.concatenate((gru_out[:, -1, :128], gru_out[:, 0, 128:]), axis=-1)
h_final.shape

(1, 256)

In [444]:
# output = model.linear(h_final)
h_final_reshaped = h_final.reshape(-1, model.linear.weight.shape[1])
# Calculate the output
output = model.linear(torch.tensor(h_final_reshaped, dtype=torch.float32))

In [445]:
output.shape

(2, 8)

In [446]:
# LRP backward
target_class = np.argmax(output, 1)
rel_output = np.zeros(output.shape)
rel_output[:, target_class] = output[:, target_class]
rel_h_final = model.linear.attr_backward(rel_output)

In [447]:
rel_gru_out = np.zeros(gru_out.shape)

In [448]:
rel_gru_out.shape, rel_h_final.shape

((1, 50, 256), (2, 128))

In [449]:
rel_gru_out[:, -1, :128].shape, rel_h_final[:, :].shape

((1, 128), (2, 128))

In [450]:
rel_gru_out[:, 0, 128:].shape, rel_h_final[:, :].shape

((1, 128), (2, 128))

In [451]:
rel_gru_out.shape

(1, 50, 256)

In [452]:
rel_h_final.shape

(2, 128)

In [453]:
# rel_gru_out[:, -1, :60] = rel_h_final[:, :60]
# rel_gru_out[:, 0, 60:] = rel_h_final[:, 60:]
rel_gru_out[:, -1, :128] = rel_h_final[:, :64].reshape((1,128))
rel_gru_out[:, 0, 128:] = rel_h_final[:, 64:].reshape((1,128))

In [454]:
rel_gru_out.shape

(1, 50, 256)

In [455]:
# rel_gru_out.reshape((25, 50, 256))

In [457]:
rel_embeddings = model.gru.attr_backward(rel_gru_out)

In [462]:
rel_embeddings.shape

(1, 50, 768)

In [463]:
print("Relevance Scores:")
for i, rel_w in enumerate(rel_embeddings.sum(axis=-1).squeeze()):
    print("    {}: {:.4f}".format(x_test[0][i], rel_w))

Relevance Scores:
    0: 15498.7839
    0: 1261.9948
    0: 139.4122
    0: 1.8828
    0: -20.3136
    0: -17.0793
    0: 10.9958
    0: 134.4200
    0: 721.3337
    0: 3720.2684
    0: 20111.8979
    0: 116299.7968
    0: 726128.7392
    0: 4936352.8957
    0: 36893369.0947
    0: 306735197.7527
    0: 2879440597.8186
    0: 31115412175.1212
    0: 397305058627.3771
    0: 6222788654892.7207
    0: 126668723281428.0938
    0: 3709299365651620.0000
    0: 196180105358115360.0000
    0: 57776215301763399680.0000
    0: -3974700579458029453312.0000
    0: 129836084008085471887360.0000
    0: -2763487398178540637650944.0000
    0: 43410916183202776899125248.0000
    0: -538390243056092696381751296.0000
    0: 5496392450383243154745720832.0000
    0: -47540299564969617153706688512.0000
    0: 355796368926826496715145609216.0000
    0: -2341329381828245561735039156224.0000
    0: 13713908501897484883422431150080.0000
    0: -72207880782022820902143868796928.0000
    0: 344627748410059877365